In [1]:
import pandas as pd

In [2]:
bitcoin_path = 'data/bitcoin_price.csv'
ethereum_path = 'data/ethereum_price.csv'
neo_path = 'data/neo_price.csv'
iota_path = 'data/iota_price.csv'

bitcoin_df = pd.read_csv(bitcoin_path)
ethereum_df = pd.read_csv(ethereum_path)
neo_df = pd.read_csv(neo_path)
iota_df = pd.read_csv(iota_path)

In [3]:
bitcoin_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",11231.8,11958.5,11231.80,11403.7,"9,926,540,000","189,536,000,000"
1,"Feb 19, 2018",10552.6,11273.8,10513.20,11225.3,"7,652,090,000","178,055,000,000"
2,"Feb 18, 2018",11123.4,11349.8,10326.00,10551.8,"8,744,010,000","187,663,000,000"
3,"Feb 17, 2018",10207.5,11139.5,10149.40,11112.7,"8,660,880,000","172,191,000,000"
4,"Feb 16, 2018",10135.7,10324.1,9824.82,10233.9,"7,296,160,000","170,960,000,000"


In [4]:
ethereum_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",943.57,965.26,892.95,895.37,"2,545,260,000","92,206,500,000"
1,"Feb 19, 2018",921.67,957.78,921.55,943.87,"2,169,020,000","90,047,700,000"
2,"Feb 18, 2018",973.35,982.93,915.45,923.92,"2,567,290,000","95,077,100,000"
3,"Feb 17, 2018",944.75,976.60,940.75,974.12,"2,525,720,000","92,264,000,000"
4,"Feb 16, 2018",934.79,950.00,917.85,944.21,"2,369,450,000","91,272,100,000"


In [5]:
neo_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",137.95,142.01,127.86,128.36,"239,238,000","8,966,490,000"
1,"Feb 19, 2018",127.23,138.52,126.70,137.47,"201,551,000","8,270,290,000"
2,"Feb 18, 2018",136.40,137.95,124.32,127.38,"207,624,000","8,866,100,000"
3,"Feb 17, 2018",131.03,139.80,129.42,136.75,"286,385,000","8,516,620,000"
4,"Feb 16, 2018",123.02,130.62,119.94,128.85,"194,878,000","7,996,380,000"


In [6]:
iota_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",2.09,2.12,1.88,1.91,"49,930,400","5,800,510,000"
1,"Feb 19, 2018",1.99,2.14,1.99,2.09,"32,581,400","5,521,610,000"
2,"Feb 18, 2018",2.20,2.21,1.98,1.98,"43,653,800","6,121,620,000"
3,"Feb 17, 2018",2.10,2.21,2.08,2.20,"42,149,700","5,842,140,000"
4,"Feb 16, 2018",2.12,2.18,2.00,2.10,"44,770,200","5,894,520,000"
